## Pima Indians Diabetes Modeling
In the [previous notebook](https://www.kaggle.com/michaelchen1116/pima-indians-diabetes-eda): 

* We conducted exploratory data analysis on the `diabetes` dataset through the process of **univariate analysis**, **bivariate analysis**, **missingness analysis**. 
* We identified zero entries as missing values in different columns and imputed those values using the existing data from those columns. 
* We selected columns that best explain the target variables and have low residuals relative to their counterpart (either original or bins that we created by cutting original columns).

This notebook contains the modeling step for the diabetes analysis. It presents the process of preprocessing, creating models, parameter tuning, and comparing models using different evaluation metrics.

In [ ]:
# data analysis, wrangling and preprocessing
import numpy as np
import pandas as pd
import random

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# statistical modeling
import statsmodels.api as sm

# modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# model evaluation
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix 

### Loading Data
We start by loading the dataset from the previous anaysis into Pandas DataFrame.

In [ ]:
diabetes = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
diabetes.columns

Here is a brief description of each column:
* `Pregnancies`: number of times pregnant
* `Glucose`: blood plasma glucose concentration a 2 hours in an oral glucose tolerance test
* `BloodPressure` : Diastolic blood pressure with a unit of  mm Hg
* `SkinThickness` : triceps skin fold thickness with a unit of mm
* `Insulin` : 2-Hour serum insulin with a unit of mu U/ml
* `BMI` : body mass index, which is calculated by weight in kg/(height in m)^2
* `DiabetesPedigreeFunction` : a synthesis of the diabetes mellitus history in relatives 
and the genetic relationship of those relatives to the subject
* `Age` : age of the patient in years
* `Outcome` : whether the patient has diabetes or not, which is our target variable that we are trying 
to predict

### Modeling approach
We plan to use **logistic regression**, **random forest**, and **gradient boosting** to model whether a given patient in the dataset is classified as having diabetes or not. We also plan to go through the process of hyperparameter tuning for each model using GridSearch to obtain the best result before we compare the results from different models using our evaluation metrics.

#### Preprocessing for modeling
We will go through the process for the completeness of the analysis. Here are several things we have to do to reach the resulting dataset from the the [previous notebook](https://www.kaggle.com/michaelchen1116/pima-indians-diabetes-eda):
* Replace zeros by imputing existing values in that column randomly. 
* Next, we will cut some columns we selected in the previous notebook into bins since the original columns has either larger residual or lower R-square.

We identify zero values in `BloodPressure`, `Insulin`, and `SkinThickness` as missing values. As a result, we decided to handle them by imputing existing values in that column randomly. 

In [ ]:
random.seed(0)
diabetes['BloodPressure']=diabetes['BloodPressure'].replace(0, random.choice(diabetes[diabetes.BloodPressure != 0 ]["BloodPressure"].tolist()))
diabetes['SkinThickness']=diabetes['SkinThickness'].replace(0, random.choice(diabetes[diabetes.SkinThickness != 0 ]["SkinThickness"].tolist()))
diabetes['Insulin']=diabetes['Insulin'].replace(0, random.choice(diabetes[diabetes.Insulin != 0 ]["Insulin"].tolist()))

We will cut these columns into bins since the original columns has either larger residual or lower R-square:
* `Insulin`
* `BMI`
* `DiabetesPedigreeFunction`

In [ ]:
diabetes['Insulin_group'] = pd.cut(diabetes['Insulin'], bins = 3, labels=False)
diabetes['BMI_group'] = pd.cut(diabetes['BMI'], bins = 5, labels=False)
diabetes['pedigree_group'] = pd.cut(diabetes['DiabetesPedigreeFunction'], bins = 3, labels=False)

Only select columns we chose to put into the model. The process of feature selection can be found in the [previous notebook](https://www.kaggle.com/michaelchen1116/pima-indians-diabetes-eda):

In [ ]:
diabetes = diabetes[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
       'Insulin_group', 'BMI_group', 'pedigree_group', 'Outcome']]

We want to use the heatmap to see if there are any features that are highly correlated with other features. If there is such feature, we have to drop them since they provide redundant information for our model.

In [ ]:
sns.heatmap(diabetes.iloc[:,:-1].corr())
plt.xticks(rotation=45)
plt.title("Heapmap of Selected Feautures")

There are not two columns that are significantly correlated with each other. We can proceed to the modeling step.

To avoid duplicate code, we decided to create `X` (independent variables) and `y` (dependent variable) in advance and splitting the dataset into training and testing sets using `train_test_split` for the purpose of getting more accurate results for the model evaluation process. 

In [ ]:
X = diabetes.drop(columns=['Outcome'])
X = sm.add_constant(X)
y = diabetes['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#### Logistic Regression
Logistic regression is perfect for a binary classification problem like this one where we try to predict whether the patient has diabetes or not. It predicts the probability of each outcome and we can round it to 0 and 1. 

Here is a brief description of the parameters we are trying to tune:
* **Penalty**: the type of regularization between 'L1' and 'L2'. 'L1' is the Lasso regression, which adds a "squared-magnitude" of coefficient as penalty term in the loss function, while 'L2' is the Ridge regression, which adds “absolute value of magnitude” of coefficient as penalty term to the loss function.
* **C**: the trade-off parameter of logistic regression that determines the strength of the regularization.

In [ ]:
log_reg = LogisticRegression()

penalty = ['l1','l2']
C = np.linspace(0, 5, 10)
hyperparameters = dict(C=C, penalty=penalty)
# Create Grid Search using 5-fold cross validation
clf = GridSearchCV(log_reg, hyperparameters, cv = 5, verbose=0)
best_logic_model = clf.fit(X_train, y_train)

In [ ]:
print('Best Penalty:', best_logic_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_logic_model.best_estimator_.get_params()['C'])

To avoid redundant code, we created a function called `model_performance` that generates the number of true negatives, true positives, false negatives, and false positives. It has two parameters: `name` (the name of the model) and `prediction` (the prediction made by the given model). It creates a dictionary with the metrics we need to build a table later on to compare different models.

In [ ]:
new_dict = {}
def model_performance(name, prediction):
    TN = confusion_matrix(y_test, prediction)[1,1]
    TP = confusion_matrix(y_test, prediction)[0,0]
    FN = confusion_matrix(y_test, prediction)[1,0]
    FP = confusion_matrix(y_test, prediction)[0,1]
    new_dict[name] = [TN, TP, FN, FP]
    return new_dict

We use the `model_performance` function that we just created to get the metrics.

In [ ]:
# performing predictions on the test datdaset 
yhat = best_logic_model.predict(X_test) 
prediction = list(map(round, yhat))
# getting model performance data for model evalation and comparison 
model_performance('Logistic', prediction)

#### Random Forest
Random forest is a collection of decision trees and each tree is independent of others. The final call depends on the majority of the decision. 

Here is a brief description of the parameters we are trying to tune:
* **n_estimators**: number of trees in the forest
* **max_depth**: the depth of each tree in the forest
* **min_samples_split**: the minimum number of samples required to split an internal node

In [ ]:
rf = RandomForestClassifier()

n_estimators = [16,32,64,128]
max_depth = np.linspace(1, 10, 10, endpoint=True)
min_samples_split = np.linspace(0.1, 1.0, 10, endpoint=True)
hyperparameters = dict(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
# Create Grid Search using 3-fold cross validation
clf = GridSearchCV(rf, hyperparameters, cv = 3, verbose=0)
best_rf_model = clf.fit(X_train, y_train)

In [ ]:
print('Best n_estimator:', best_rf_model.best_estimator_.get_params()['n_estimators'])
print('Best max_depth:', best_rf_model.best_estimator_.get_params()['max_depth'])
print('Best min_samples_split:', best_rf_model.best_estimator_.get_params()['min_samples_split'])

We want to see what are some features that play an important part in deciding how to classify patients in this dataset into two distinct groups. 

In [ ]:
rf = RandomForestClassifier(n_estimators=16, max_depth=8, min_samples_split=0.1)
rf_model = rf.fit(X_train, y_train)
feature_imp = pd.Series(rf_model.feature_importances_,index=X.columns).sort_values(ascending=False)
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()

As we can see in the chart above, `Glucose` plays a significant role comparing with other features in deciding where to split the tree into different branches followed by `Pregnancies` and `BMI_group`. This makes sense because according to [this source](https://www.niddk.nih.gov/health-information/diabetes/overview/what-is-diabetes), diabetes normally occurs glucose, also called blood sugar, is too high. Also, `BMI_group` and `Pregnancies` both reflect the weight of a patient, which is a crucial factor when determining whether a patient has diabetes or not.

It is surprising to us that Insulin doesn't play much of a role here because it helps glucose from food get into your cells to be used for energy. The reason might be that we cut the original `Insulin` column into three evenly distributed bins instead of cutting them into bins according to medical ranges that define different levels of insulin.

Performing predictions on the test dataset and using the function we created earlier to get model performance data for model evaluation and comparison.

In [ ]:
yhat = best_rf_model.predict(X_test) 
prediction = list(map(round, yhat)) 
model_performance('Random Forest', prediction)

#### Gradient Boost Model
Gradient boosting relies on the intuition that the best possible next model, when combined with the previous model, minimizes the overall prediction error.

Here is a brief description of the parameters we are trying to tune:
* **learning_rate**: the step size of optimization algorithm moving towards the objective
* **n_estimators**: number of trees in the forest
* **max_depth**: the depth of each tree in the forest
* **min_samples_split**: the minimum number of samples required to split an internal node

In [ ]:
gb = GradientBoostingClassifier()

learning_rate = [1, 0.5, 0.25, 0.1]
n_estimators = [1,2,4,8,16,32,64]
max_depth = np.linspace(1, 5, 5, endpoint=True)
min_samples_split = np.linspace(0.1, 0.5, 5, endpoint=True)
hyperparameters = dict(learning_rate=learning_rate,n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
# Create Grid Search using 3-fold cross validation
clf = GridSearchCV(gb, hyperparameters, cv = 3, verbose=0)
best_gb_model = clf.fit(X_train, y_train)

In [ ]:
print('Best learning_rate:', best_gb_model.best_estimator_.get_params()['learning_rate'])
print('Best n_estimator:', best_gb_model.best_estimator_.get_params()['n_estimators'])
print('Best max_depth:', best_gb_model.best_estimator_.get_params()['max_depth'])
print('Best min_samples_split:', best_gb_model.best_estimator_.get_params()['min_samples_split'])

Performing predictions on the test dataset and using the function we created earlier to get model performance data for model evaluation and comparison.

In [ ]:
yhat = best_gb_model.predict(X_test) 
prediction = list(map(round, yhat))
model_performance('Gradient Boosting', prediction)

#### Model performance comparison
Since this is a classification problem, we decided to use corresponding metrics to compare the model performance (listed as follows).
* **accuracy**: the proportion of predictions that are correct
* **sensitivity** (true positive rate): the proportion of true observations that are correctly predicted by the model as being true.
* **specificity** (true negative rate): the proportion of false observations that are correctly predicted by the model as being false.

To avoid repetitive code, we decided to create a funcction that calculates those different metrics that we just mentioned. 

In [ ]:
def evaluation_metric(aDict):
    for name in aDict:
        TN = aDict[name][0]
        TP = aDict[name][1]
        FN = aDict[name][2]
        FP = aDict[name][3]
        accuracy = np.round((TN+TP)/ (TN+TP+FN+FP), 4)
        # sensitivity is calculated by the true positives over the sum of the true positives and false negatives
        sensitivity = np.round((TP)/ (TP+FN), 4)
        # specificity is calculated by the true negatives over the sum of the true negatives and the false positives
        specificity = np.round((TN)/ (TN+FP), 4)
        aDict[name].append(accuracy)
        aDict[name].append(sensitivity)
        aDict[name].append(specificity)
    return aDict

##### Create a table using the dictionary
The table includes performance data and evaluation metrics for each model.
* First, we count the number of test cases in each model
* Then, we want to transform the numbers in the confusion matrix into percentages.

In [ ]:
# count the sum of test cases
sum_list = []
for aList in new_dict.values():
    aSum = 0
    for i in np.arange(4):
        aSum += aList[i]
    sum_list.append(aSum)
sum_list

In [ ]:
for aList in new_dict.values():
    for j in np.arange(4):
        aList[j] = np.round(aList[j]/254, 3)

In [ ]:
model_comparison = pd.DataFrame.from_dict(evaluation_metric(new_dict))
model_comparison = model_comparison.rename(index={0: "True Negative", 1: "True Positive", 2: "False Negative",3:'False Positive',
    4:'Accuracy',5:'Sensitivity',6:'Specificity'})
model_comparison

According to the table above, we can see that logistic regression seems to have the most accuracy in terms of predicting whether a patient in this dataset is classified as having diabetes or not. There is not much of a difference in model performance after the process of parameters tuning.

**Logistic Regression** algorithm has the highest true positive rate. This means that **Logistic Regression** is the most useful in correctly predicting whether a patient should be classified as having diabetes for this dataset.

**Logistic Regression** has the highest sensitivity. This means that Logistic Regression correctly generates a positive result for people who have diabetes more often than other models. Also, we have to note that a model with highly sensitive will flag almost everyone who has the disease and not generate many false-negative results. In this case, around 78% does not raise this red flag of being too high. 

**Logistic Regression** has the highest specificity. This means that Logistic Regression correctly generates a negative result for people who don't have diabetes. Similar to sensitivity, a high specificity will raise a red flag because it means the model will classify everyone in the dataset as not having diabetes. In our Logistic Regression model, around 70% of specificity does not raise this red flag. 

Normally, we will choose the model that correctly identifying patients without diabetes was more important than correctly identifying patients with diabetes. Ane we will choose the model that correctly identifying patients with diabetes was more important than correctly identifying patients without diabetes. In this case, since **Logistic Regression** has the highest sensitivity and specificity at the same time. There is no doubt that it is our best model when we try to predict whether patients in this dataset have diabetes or not.

##### Functions that help us generate the ROC-AUC curve 
* `no_skill_prediction` generates a plot with a line that shows when the true positive equals the false positive rate.
* `graph_roc_auc` generates ROC-AUC curve for a given model

In [ ]:
def no_skill_prediction():
    ns_probs = [0 for _ in range(len(y_test))]
    ns_auc = roc_auc_score(y_test, ns_probs)
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')

def graph_roc_auc(model, name):
    # predict probabilities
    lr_probs = model.predict_proba(X_test)
    lr_probs = lr_probs[:, 1]
    # calculate score
    lr_auc = roc_auc_score(y_test, lr_probs)
    # print score
    print(name + ': ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    # plot the roc curve for the model
    plt.plot(lr_fpr, lr_tpr, marker='.', label=name)
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()

##### Produce the ROC AUC curves for all three models on the same graph
We generate ROC AUC curves for all three models on the same graph using the function we created earlier.

In [ ]:
no_skill_prediction()
model = [best_logic_model, best_rf_model, best_gb_model]
name = ['Logistic','Random Forest','Gradient Boosting']
for i in range(len(model)):
    graph_roc_auc(model[i], name[i])
plt.title('ROC AUC comparison')

As shown in the ROC AUC curve above, the y-axis is the true positive rate (sensitivity), while the x-axis is the false positive rate (1- specificity). The true positive rate tells us the proportion of patients in this dataset that identified as having diabetes were correctly classified. The false-positive rate tells us the proportion of patients in this dataset that identified as not having diabetes were correctly classified and are false positive. 

The line for each model as shown in orange, green, and red illustrates the trade-off between we correctly classified all of the patients in this dataset that have diabetes and incorrectly classified all of the patients in this dataset that don't have diabetes.

The area under the curve (AUC) makes it easier to compare one ROC curve to another. They were shown above the graph. The graph and the AUC confirm that we should choose logistic regression. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fd4a356b-76ac-4ad6-9eb0-790688a80ec2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>